# LangGraph: Agent Executor

[LangGraph](https://python.langchain.com/docs/langgraph) \
[Tutorial on Youtube](https://www.youtube.com/watch?v=9dXp5q3OFdQ&list=PLfaIDFEXuae16n2TWUkKq5PgJ0w6Pkwtg&index=3&ab_channel=LangChain) \
[Tools: Google Search](https://python.langchain.com/docs/integrations/tools/google_search)

In [1]:
import os
import sys

from dotenv import load_dotenv

sys.path.append(os.path.abspath(os.pardir))
load_dotenv('../.env')

True

In [3]:
assert os.environ['LANGCHAIN_TRACING_V2']

In [2]:
from langchain_google_vertexai import VertexAI

llm = VertexAI(model_name='gemini-pro')

In [37]:
from langchain.tools import Tool
from langchain_community.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper(k=5)


def top5_results(query) -> str:
    return search.results(query, 5)


tools = [
    Tool(
        name="Google Search Snippets",
        description="Search Google for recent results.",
        func=search.run,
        # func=top5_results,
    )
]

In [38]:
from langchain import hub

prompt = hub.pull("hwchase17/react")
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [39]:
from langchain.agents import create_react_agent

agent_runnable = create_react_agent(llm, tools, prompt)

In [40]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

In [41]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executor = ToolExecutor(tools)


def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}


def execute_tools(data):
    agent_action = data["agent_outcome"]
    output = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(output))]}


def should_continue(data):
    if isinstance(data["agent_outcome"], AgentFinish):
        return "end"
    else:
        return "continue"

In [42]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(AgentState)

workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)
workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END,
    },
)

workflow.add_edge("action", "agent")

app = workflow.compile()

In [43]:
inputs = {"input": "what is the weather in sf", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'agent_outcome': AgentAction(tool='Google Search Snippets', tool_input='what is the weather in sf', log='Action: Google Search Snippets\nAction Input: what is the weather in sf')}
----
{'intermediate_steps': [(AgentAction(tool='Google Search Snippets', tool_input='what is the weather in sf', log='Action: Google Search Snippets\nAction Input: what is the weather in sf'), "Be prepared with the most accurate 10-day forecast for San Francisco, CA with highs, lows, chance of precipitation from The Weather Channel and Weather.com. Hourly Weather · 1 AM 51°. rain drop 56% · 2 AM 51°. rain drop 49% · 3 AM 51°. rain drop 49% · 4 AM 51°. rain drop 63% · 5 AM 51°. rain drop 49% · 6 AM 51°. Be prepared with the most accurate 10-day forecast for Inverness, CA with highs, lows, chance of precipitation from The Weather Channel and Weather.com. Current conditions at. SAN FRANCISCO DOWNTOWN (SFOC1). Lat: 37.77056°NLon: 122.42694°WElev: 150.0ft. NA. 52°F. 11°C. Humidity, 100%. Wind Speed, NA NA MPH. To

In [44]:
output = app.invoke(inputs)
output

{'input': 'what is the weather in sf',
 'chat_history': [],
 'agent_outcome': AgentFinish(return_values={'output': '52 degrees Fahrenheit with 100% humidity and a chance of rain'}, log='Final Answer: 52 degrees Fahrenheit with 100% humidity and a chance of rain'),
 'intermediate_steps': [(AgentAction(tool='Google Search Snippets', tool_input='what is the weather in sf', log='Action: Google Search Snippets\nAction Input: what is the weather in sf'),
   "Be prepared with the most accurate 10-day forecast for San Francisco, CA with highs, lows, chance of precipitation from The Weather Channel and Weather.com. Hourly Weather · 1 AM 52°. rain drop 56% · 2 AM 52°. rain drop 49% · 3 AM 51°. rain drop 49% · 4 AM 51°. rain drop 63% · 5 AM 51°. Be prepared with the most accurate 10-day forecast for Inverness, CA with highs, lows, chance of precipitation from The Weather Channel and Weather.com. Current conditions at. SAN FRANCISCO DOWNTOWN (SFOC1). Lat: 37.77056°NLon: 122.42694°WElev: 150.0ft. N

In [47]:
output['agent_outcome'].return_values['output']

'52 degrees Fahrenheit with 100% humidity and a chance of rain'